In [1]:
!pip install recipe-scrapers
!pip install yagmail

In [41]:
debug_level = 2

In [3]:
from recipe_scrapers import scrape_me
from datetime import datetime

In [4]:
def debug1(debug_string):
    if debug_level >= 1:
        print('\nDebug1:\n' + debug_string)
    return

def debug2(debug_string):
    if debug_level >= 2:
        print('\nDebug2:\n' + debug_string)
    return

def debug3(debug_string):
    if debug_level >= 3:
        print('\nDebug3:\n' + debug_string)
    return

def debug4(debug_string):
    if debug_level >= 4:
        print('\nDebug4:\n' + debug_string)
    return

In [5]:
import getpass

password = getpass.getpass(prompt = 'Enter the password')

Enter the password ·········


In [6]:

# datetime object containing current date and time
right_now = datetime.now()

# dd/mm/YY H:M:S
dt_string = right_now.strftime("%d/%m/%Y %H:%M:%S")
debug4('Date and Time: ' + dt_string)

In [66]:
def markup_recipe(rTitle,rURL,rImage,rIngredients,rInstructions):

    now = datetime.now()
    rDate = now.strftime("%m/%d/%Y %H:%M")
    
    recipe_markup = 'Title: ' + rTitle + '<br>\nDate: ' + rDate + '\nCategory: Recipes<br>\n'
    recipe_markup = recipe_markup + '__[' + rTitle + ']('+ rURL + ')__<br>\n'
    recipe_markup = recipe_markup + """<img src='""" + rImage + """'><br>\n"""
    rIngredients = rIngredients.replace('\n','<br>\n')
    recipe_markup = recipe_markup + '> ' + rIngredients + '<br>\n\n'
    rInstructions = rInstructions.replace('\n', '<br>\n\n1. ')
    recipe_markup = recipe_markup + '1. ' + rInstructions
    debug2(recipe_markup)
    return(recipe_markup)
   
    

In [69]:
# Different ways to scrape during testing Comment out all but one

# scraper=scrape_me('https://cooking.nytimes.com/recipes/1016595-hamburgers-diner-style')
# scraper=scrape_me('https://cooking.nytimes.com/recipes/1022421-bulgogi-eggplant')
# scraper=scrape_me('https://www.seriouseats.com/fish-fragrant-eggplants-sichuan-braised-eggplant-with-garlic-ginger-and-chilies')

scraper = scrape_me(input('Recipe URL:'))

Recipe URL: https://cooking.nytimes.com/recipes/1016595-hamburgers-diner-style


In [70]:
# Consolidate the recipe elements

recipe_ingredients =scraper.ingredients()

# Add a newline after each ingredient and convert the list to a string
recipe_ing_temp = [sub + '\n' for sub in recipe_ingredients]
recipe_ingredients = ' '.join(recipe_ing_temp)

recipe_title = scraper.title()
recipe_instructions = scraper.instructions()
recipe_image = scraper.image()
recipe_URL = scraper.url
recipe_site_name = scraper.site_name()

formatted_recipe = markup_recipe(recipe_title, recipe_URL, recipe_image, recipe_ingredients, recipe_instructions)
debug3(formatted_recipe)


Debug2:
Title: Hamburgers (Diner Style)<br>
Date: 08/19/2021 09:58
Category: Recipes<br>
__[Hamburgers (Diner Style)](https://cooking.nytimes.com/recipes/1016595-hamburgers-diner-style)__<br>
<img src='https://static01.nyt.com/images/2019/07/14/multimedia/sifton-hamburgers/sifton-hamburgers-mediumThreeByTwo440-v3.jpg'><br>
> 1/2 teaspoon neutral oil, like canola, or a pat of unsalted butter<br>
 2 pounds ground chuck, at least 20 percent fat<br>
 Kosher salt and black pepper to taste<br>
 8 slices cheese (optional)<br>
 8 soft hamburger buns, lightly toasted<br>
 Lettuce leaves, sliced tomatoes and condiments, as desired<br>
<br>

1. Add oil or butter to a large cast-iron or stainless-steel skillet and place over medium heat. Gently divide ground beef into 8 small piles of around 4 ounces each, and even more gently gather them together into orbs that are about 2 inches in height. Do not form patties.<br>

1. Increase heat under skillet to high. Put half the orbs into the skillet with 

In [71]:
# Change the current working directory to the Website content directory
debug4("Original working directory: " + os.getcwd())
os.chdir('./Recipes_Website/content')
debug4("Website content directory: " + os.getcwd())

with open(recipe_title + '.md', 'w') as writer:
    writer.write(formatted_recipe)
    writer.close()
    
# Restore the CWD to the original
os.chdir('../../')
debug4("Restored to Original working directory: " + os.getcwd())


In [8]:
# Prettify the message payload

message = recipe_title + '\n\n' + recipe_URL + '\n\n '

recipe_instructions = recipe_instructions.replace('\n','\n\n')

# Assemble the full message
message = message + recipe_ingredients + '\n' + recipe_instructions

# Is gmail filtering messages with offensive phrases? 
# if you change Asian to something different, it works.
# See the yagmail example below for a different substitution

message = message.replace('Asian', 'slender')

# email setup and code from tutorial at https://realpython.com/python-send-email/

sender_email = "shapim919@gmail.com"  # Enter your address
receiver_email = "iironmiike1@gmail.com"  # Enter receiver address

  # Simplified way  to send the message (from the same tutorial as above)

import yagmail
yag = yagmail.SMTP(sender_email, password)

# remove 'Asian eggplant' from the recipe ingredients and it works. Go figure
# (this is when using the Eggplant Bulgogi recipe)

ings = recipe_ingredients.replace('Asian eggplant','slender eggplant')

mixed_contents = [recipe_title, '\n', recipe_URL,'\n',recipe_image, '\n', ings, '\n', recipe_instructions]

debug3(" Email payload for yagmail:\n" + '\n'.join(str(items) for items in mixed_contents))

yag.send(receiver_email, recipe_title, mixed_contents)

Recipe URL: https://www.seriouseats.com/fish-fragrant-eggplants-sichuan-braised-eggplant-with-garlic-ginger-and-chilies


ERROR:extruct._extruct:Failed to extract json-ld, raises Expecting value: line 1 column 1 (char 0)
Traceback (most recent call last):
  File "c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\extruct\_extruct.py", line 108, in extract
    output[syntax] = list(extract(document, base_url=base_url))
  File "c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\extruct\jsonld.py", line 25, in extract_items
    return [
  File "c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\extruct\jsonld.py", line 28, in <listcomp>
    if items for item in items if item
  File "c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\extruct\jsonld.py", line 38, in _extract_items
    data = jstyleson.loads(HTML_OR_JS_COMMENTLINE.sub('', script), strict=False)
  File "c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\jstyleson.py", line 123, in loads
    return json.loads(dispose(text), **kwargs)

{}

In [16]:
scraper.

Signature: scraper.nutrients()
Docstring: <no docstring>
Source:   
    def nutrients(self):
        raise NotImplementedError("This should be implemented.")
File:      c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\recipe_scrapers\_abstract.py
Type:      method


In [10]:
scraper.ingredients??

Signature: scraper.ingredients()
Docstring: <no docstring>
Source:   
    def ingredients(self):
        ingredients = self.soup.findAll("li", {"class": "ingredient"})
        return [normalize_string(ingredient.get_text()) for ingredient in ingredients]
File:      c:\users\iiron\documents\python_workspaces\recipes\venv\lib\site-packages\recipe_scrapers\seriouseats.py
Type:      method
